In [1]:
import dask
dask.config.set({'distributed.scheduler.allowed-failures': 100});

In [3]:
import dask.distributed as  Client, LocalCluster
    
from dask import bag as db
import os
import random
from  time import sleep

ModuleNotFoundError: No module named 'LocalCluster'

In [19]:
cluster = LocalCluster(threads_per_worker=1, n_workers=4, memory_limit=400e6)
client = Client(cluster)
client

NameError: name 'LocalCluster' is not defined

In [ ]:
def multiply_by_two(x):
    sleep(0.02)
    return 2 * x

In [ ]:
N = 400
x = db.from_sequence(range(N), npartitions=N // 2)
mults = x.map(multiply_by_two)
summed = mults.sum()

In [ ]:
all_current_workers = [w.pid for w in cluster.scheduler.workers.values()]
non_preemptible_workers = all_current_workers[:2]

In [ ]:
def kill_a_worker():
    preemptible_workers = [
        w.pid for w in cluster.scheduler.workers.values()
        if w.pid not in non_preemptible_workers]
    if preemptible_workers:
        os.kill(random.choice(preemptible_workers), 15)

In [ ]:
summed = client.compute(summed)
while no summed.done():
    kill_a_worker()
    sleep(3.0)

In [ ]:
print(f"`sum(range({N}))` on cluster: {summed.result()}\t(should be {N * (N-1)})")